In [ ]:
import pymysql
pymysql.install_as_MySQLdb()

from sql import *
from stats import *
import pandas as pd
from lifelines import *

In [ ]:
from matplotlib import pyplot
%matplotlib notebook

In [ ]:
fragdata = split_data(download_frag_data())
mainzones = ['Water Villas (LG)', 'Channel (KH)', 'Blue Holes (LG)', 'Parrot Reef (LG)', 'Water Villas (KH)', 'House Reef (KH)', 
         'Blu (LG)', 'Blu Deep (LG)', 'Dive Site (LG)', 'Coral Trail (LG)', 'Anchor Point (LG)', 'Al Barakat (LG)']
controlzones = ['Blu (LG)', 'Blu Deep (LG)', 'Dive Site (LG)', 'Coral Trail (LG)', 'Anchor Point (LG)', 'Al Barakat (LG)']
disruptedzones = ['Water Villas (LG)', 'Channel (KH)', 'Blue Holes (LG)', 'Parrot Reef (LG)', 'Water Villas (KH)', 'House Reef (KH)']

main = fragdata['all'][fragdata['all']['Zone'].isin(mainzones)]
control = fragdata['all'][fragdata['all']['Zone'].isin(controlzones)]
disrupted = fragdata['all'][fragdata['all']['Zone'].isin(disruptedzones)]

# Mortality

In [ ]:
survived, dead = get_mortality()

In [ ]:
roll_window = 120
fig, axs = pyplot.subplots(2, sharex = True, figsize = [13,8])
mortality = {}
lines = {}
colors = [pyplot.cm.tab10(x) for x in np.linspace(0,1,10)] + [pyplot.cm.Accent(x) for x in np.linspace(0,1,8)]
axs[0].set_prop_cycle('color', colors)

for z in mainzones:
    zone_survived = survived[survived['Zone'] == z]
    zone_dead = dead[dead['Zone'] == z]
    mortality[z] = zone_dead['Median'].value_counts() / (zone_dead['Median'].value_counts() + zone_survived['Median'].value_counts())
    mortality[z] = mortality[z].sort_index().rolling(str(roll_window) + 'd').sum().rename(z) / roll_window * 100
    lines[z], = axs[0].plot_date(mortality[z].index, mortality[z], '-')

for z in mainzones[4:]:
    zone_survived = survived[survived['Zone'] == z]
    zone_dead = dead[dead['Zone'] == z]
    mortality[z] = zone_dead['Median'].value_counts() / (zone_dead['Median'].value_counts() + zone_survived['Median'].value_counts())
    mortality[z] = mortality[z].sort_index().rolling(str(roll_window) + 'd').sum().rename(z) / roll_window * 100
    axs[1].plot_date(mortality[z].index, mortality[z], '-', color = lines[z].get_color(), label = '_' + z)

colors = pd.read_csv('kh_temp_alert.csv', index_col = 0, parse_dates = True)
d = colors.index
lim = axs[1].get_ylim()[1]
axs[1].fill_between(d, 0, lim, where=colors['level'] == 0, facecolor='yellow', alpha=0, label = 'NOAA level 0: No Stress')
axs[1].fill_between(d, 0, lim, where=colors['level'] == 1, facecolor='yellow', alpha=0.3, label = 'NOAA level 1: Watch')
axs[1].fill_between(d, 0, lim, where=colors['level'] == 2, facecolor='orange', alpha=0.3, label = 'NOAA level 2: Warning')
axs[1].fill_between(d, 0, lim, where=colors['level'] == 3, facecolor='red', alpha=0.3)
axs[1].fill_between(d, 0, lim, where=colors['level'] == 4, facecolor='darkred', alpha=0.3)


axs[0].legend(loc='lower left', ncol=6, bbox_to_anchor=(0., 1.03, 1., .102), mode="expand", borderaxespad=0.)
axs[1].legend(loc='lower left', ncol=6, bbox_to_anchor=(0., 1.03, 1., .102), mode="expand", borderaxespad=0.)
axs[0].set(ylabel='Mortality rate (%)')
axs[1].set(ylabel='Mortality rate (%)')
axs[0].spines['top'].set_visible(False)
axs[0].spines['right'].set_visible(False)
axs[1].spines['top'].set_visible(False)
axs[1].spines['right'].set_visible(False)
axs[0].text(-0.05, -0.0, 'a', transform=axs[0].transAxes, size=12, weight='bold')
axs[1].text(-0.05, -0.0, 'b', transform=axs[1].transAxes, size=12, weight='bold')

# Global survival results

In [ ]:
fig, ax = pyplot.subplots(figsize=[12, 8])

df0 = fragdata['all']
df1 = control[control['Type'] == 'Acropora']
df2 = control[control['Type'] == 'Pocillopora']
df3 = disrupted[disrupted['Type'] == 'Acropora']
df4 = disrupted[disrupted['Type'] == 'Pocillopora']

kmf0 = KaplanMeierFitter()
kmf0.fit(df0['T'] / 30.437, event_observed=df0['E'], label = 'All')
kmf1 = KaplanMeierFitter()
kmf1.fit(df1['T'] / 30.437, event_observed=df1['E'], label = 'Control Acropora')
kmf2 = KaplanMeierFitter()
kmf2.fit(df2['T'] / 30.437, event_observed=df2['E'], label = 'Control Pocillopora')
kmf3 = KaplanMeierFitter()
kmf3.fit(df3['T'] / 30.437, event_observed=df3['E'], label = 'Disrupted Acropora')
kmf4 = KaplanMeierFitter()
kmf4.fit(df4['T'] / 30.437, event_observed=df4['E'], label = 'Disrupted Pocillopora')

kmf0.plot_survival_function(ax=ax, color = 'black', ci_alpha = 0.1)
kmf1.plot_survival_function(ax=ax, color = 'cornflowerblue', ci_alpha = 0.1)
kmf3.plot_survival_function(ax=ax, color = 'navy', ci_alpha = 0.1)
kmf2.plot_survival_function(ax=ax, color = 'lightcoral', ci_alpha = 0.1)
kmf4.plot_survival_function(ax=ax, color = 'firebrick', ci_alpha = 0.1)

from lifelines.plotting import add_at_risk_counts
add_at_risk_counts(kmf0, ax=ax)

ax.set_ylim(bottom=0)
ax.set_xlabel('Timeline (months)')
ax.set_ylabel('Survival Rate')
ax.spines['top'].set_visible(False)
ax.spines['right'].set_visible(False)
ax.legend(prop={'size': 12})


# Death / bleaching stats

In [ ]:
request = """SELECT Fragments.Type AS Sp, Status.Type, COUNT(Status.FragmentId) FROM (
        SELECT FragmentId, MIN(Date) AS death FROM Status WHERE Type IN ('Dead Coral', 'Fallen') GROUP BY FragmentId
        ) A INNER JOIN Status ON A.FragmentId = Status.FragmentId AND A.death = Status.Date
        INNER JOIN Fragments ON Fragments.FragmentId = A.FragmentID
        INNER JOIN FSFrames ON Fragments.Tag = FSFrames.Tag
        WHERE Fragments.Type IN ('Acropora', 'Pocillopora')
        GROUP BY Sp, Type"""

db = OpenMydb()
cur = db.cursor()
cur.execute(request)
for c in cur.fetchall():
    print('{}, {}: {}'.format(c[0], c[1], c[2]))

In [ ]:
bleached = get_bleached_data()
initialstress = bleached[bleached['BleachDelay'] <= 15]
print(initialstress['Outcome'].value_counts()/initialstress.shape[0])

laterstress = bleached[bleached['BleachDelay'] > 15]
print(laterstress['Zone'].value_counts()/laterstress.shape[0])

# Cox Regression (control)

In [ ]:
df = control.copy()
df['Size'] = 0.3296*df['Size']
df['tdelta'] = (df['Transplanted'] - df['StartDate']).dt.days <= 90
df = df[df['tdelta']]

X = df[['T', 'E', 'Size', 'Depth', 'Type']]
X['Bottom'] = 1*(df['Elev'] == 'A')
X['Sandy'] =1*df['Sandy']

cph = CoxPHFitter()
cph.fit(X, duration_col='T', event_col='E', formula="C(Type)*np.log(Depth)+C(Type)*Size+Sandy:Bottom", robust=True)

cph.print_summary()

In [ ]:
p_, d_, pd_, s_, ps_, sb_ = cph.params_
a_ = -p_
d = np.arange(1, 18, 0.1)
s1, s3 = 3.3, 4.7
fig, ax = pyplot.subplots(figsize = [12,9])

ax.plot(d, pd.Series(1, index = d), '--', label = 'Baseline', color = 'gray')
ax.plot(d, d**(d_)*np.exp(s1*(s_))*np.exp(sb_*0), label = 'Acropora (Small)', color = 'lightsteelblue')
ax.plot(d, d**(d_)*np.exp(s3*(s_))*np.exp(sb_*0), label = 'Acropora (Large)', color = 'slateblue')
ax.plot(d, d**(d_)*np.exp(s3*(s_))*np.exp(sb_*1), label = 'Acropora (Large, Sandy + Bottom bar)', color = 'darkblue')
ax.plot(d, np.exp(p_)*d**(d_+pd_)*np.exp(s1*(s_+ps_))*np.exp(sb_*0), label = 'Pocillopora (Small)', color = 'lightcoral')
ax.plot(d, np.exp(p_)*d**(d_+pd_)*np.exp(s3*(s_+ps_))*np.exp(sb_*0), label = 'Pocillopora (Large)', color = 'firebrick')
ax.plot(d, np.exp(p_)*d**(d_+pd_)*np.exp(s3*(s_+ps_))*np.exp(sb_*1), label = 'Pocillopora (Large, Sandy + Bottom bar)', color = 'maroon')
ax.legend()
ax.set_ylim(0, 2)
ax.set(xlabel='Depth (m)', ylabel='Hazard Ratio')
ax.spines['top'].set_visible(False)
ax.spines['right'].set_visible(False)

# Cox Regression (disrupted)

In [ ]:
long_df = get_disrupted_frag_data('heatwave')

In [ ]:
X = long_df[['FragmentId', 'Type', 'Size']].copy()
X['Start'], X['Stop']  = long_df['Start'].astype(int), long_df['Stop'].astype(int)
X['E'], X['Disrupted'] = long_df['E'].astype(int), long_df['Disrupted'].astype(int)

from lifelines import CoxTimeVaryingFitter

ctv = CoxTimeVaryingFitter(penalizer=0.0001)
ctv.fit(X, id_col="FragmentId", event_col="E", start_col="Start", stop_col="Stop", formula='C(Type)*Disrupted+C(Type):Size')
ctv.print_summary()

In [ ]:
p_, dis_, pdis_, as_, ps_ = ctv.params_
s = np.arange(2, 8, 0.01)
s1, s3 = 3.3, 4.7

fig, ax = pyplot.subplots(figsize = [12,9])

ax.plot(s, pd.Series(1, index = s), '--', label = 'Baseline', color = 'gray')
ax.plot(s, np.exp(dis_*0) * np.exp(s*as_), label = 'Acropora', color = 'lightsteelblue')
ax.plot(s, np.exp(dis_*1) * np.exp(s*as_), label = 'Acropora (Disrupted)', color = 'darkblue')
ax.plot(s, np.exp(p_) * np.exp((pdis_+dis_)*0) * np.exp(s*ps_), label = 'Pocillopora', color = 'lightcoral')
ax.plot(s, np.exp(p_) * np.exp((pdis_+dis_)*1) * np.exp(s*ps_), label = 'Pocillopora (Disrupted)', color = 'maroon')
ax.axvline(x=3.332, linestyle='--', label='Median initial fragment size', color = 'darkgreen', alpha = 0.5)
ax.fill_between(s, 0, 10, where=(s > 2.858) & (s <= 3.945), facecolor='green', alpha=0.2, label = 'Interquartile range')
ax.legend(prop={'size': 12})
ax.set_ylim(0, 10)
ax.set(xlabel='Initial fragment size (cm)', ylabel='Hazard Ratio')
ax.spines['top'].set_visible(False)
ax.spines['right'].set_visible(False)


# Growth rates

In [ ]:
livedf = get_growth_data()

In [ ]:
import seaborn as sns
from matplotlib.ticker import MultipleLocator

fig, ax = pyplot.subplots(figsize=[12,10])

sns.set_style("whitegrid")
sns.violinplot(ax=ax, x="avgrowth", y="Zone", hue='Type', data=livedf, split = True, palette="Set1", inner="quartile",
               saturation = 0.6, bw = .1, scale = 'count')
ax.set_xlabel("Growth rate (cm/year)")
ax.xaxis.set_major_locator(MultipleLocator(1))
ax.set_ylabel(None)

# Total volume

In [ ]:
live, dead, initial, fallen = get_volume_series()
acrodata = pd.DataFrame({'_Live Coral': live['Acropora'], '_Initial transplants': initial['Acropora'],
                     '_Dead Coral': -dead['Acropora'], '_Fallen Coral': -fallen['Acropora']})
pocdata = pd.DataFrame({'_Live Coral': live['Pocillopora'], '_Initial transplants': initial['Pocillopora'],
                     '_Dead Coral': -dead['Pocillopora'], '_Fallen Coral': -fallen['Pocillopora']})

acrodata.to_csv('acro_vol_data.csv')
pocdata.to_csv('poc_vol_data.csv')

In [ ]:
acrodata = pd.read_csv('acro_vol_data.csv', index_col = 0, parse_dates = True)
pocdata = pd.read_csv('poc_vol_data.csv', index_col = 0, parse_dates = True)
pocdata['Date'] = pocdata.index
acrodata['Date'] = acrodata.index
d = pocdata['Date'].values

fig, axs = pyplot.subplots(2, sharex = True, figsize = [13, 8])

axs[0].plot_date(acrodata['Date'], acrodata['_Initial transplants'], '-')
axs[0].plot_date(acrodata['Date'], (acrodata['_Dead Coral'] - acrodata['_Fallen Coral']).rename('_Dead Coral on frame'), '-')
axs[0].plot_date(acrodata['Date'], acrodata['_Live Coral'], '-')
axs[0].plot_date(acrodata['Date'], acrodata['_Dead Coral'], '-')
axs[0].fill_between(d, acrodata['_Live Coral'], acrodata['_Initial transplants'], facecolor='green', alpha=0.3, interpolate=True, label = 'Live Coral - Organic growth')
axs[0].fill_between(d, 0, acrodata['_Initial transplants'], facecolor='blue', alpha=0.3, interpolate=True, label = 'Live Coral - Initial transplants')
axs[0].fill_between(d, acrodata['_Dead Coral'] - acrodata['_Fallen Coral'], 0, facecolor='orange', alpha=0.3, interpolate=True, label = 'Dead Coral - On frame')
axs[0].fill_between(d, acrodata['_Dead Coral'], acrodata['_Dead Coral'] - acrodata['_Fallen Coral'], facecolor='red', alpha=0.3, interpolate=True, label = 'Dead Coral - Fallen')
axs[0].set(ylabel='Volume of Acropora (liters)')
axs[0].spines['top'].set_visible(False)
axs[0].spines['right'].set_visible(False)
axs[0].legend(loc=2)

axs[1].plot_date(pocdata['Date'], pocdata['_Initial transplants'], '-')
axs[1].plot_date(pocdata['Date'], (pocdata['_Dead Coral'] - pocdata['_Fallen Coral']).rename('_Dead Coral on frame'), '-')
axs[1].plot_date(pocdata['Date'], pocdata['_Live Coral'], '-')
axs[1].plot_date(pocdata['Date'], pocdata['_Dead Coral'], '-')
axs[1].fill_between(d, pocdata['_Live Coral'], pocdata['_Initial transplants'], facecolor='green', alpha=0.3, interpolate=True, label = 'Live Coral - Organic growth')
axs[1].fill_between(d, 0, pocdata['_Initial transplants'], facecolor='blue', alpha=0.3, interpolate=True, label = 'Live Coral - Initial transplants')
axs[1].fill_between(d, pocdata['_Dead Coral'] - pocdata['_Fallen Coral'], 0, facecolor='orange', alpha=0.3, interpolate=True, label = 'Dead Coral - On frame')
axs[1].fill_between(d, pocdata['_Dead Coral'], pocdata['_Dead Coral'] - pocdata['_Fallen Coral'], facecolor='red', alpha=0.3, interpolate=True, label = 'Dead Coral - Fallen')
axs[1].set(ylabel='Volume of Pocillopora (liters)')
axs[1].spines['top'].set_visible(False)
axs[1].spines['right'].set_visible(False)
axs[1].legend(loc=2)

# Add trend lines:

from sklearn.linear_model import LinearRegression

df = pd.DataFrame()
df['a'] = acrodata.loc[acrodata.index.year >= 2019]['_Live Coral']
df['p'] = pocdata.loc[pocdata.index.year >= 2019]['_Live Coral']
df['Date'] = df.index
df['RelDate'] = (df['Date'].dt.date - date(year=2019, month=1, day=1)).dt.days

x, y_a, y_p = np.array(df['RelDate']).reshape(-1, 1), np.array(df['a']).reshape(-1, 1), np.array(df['p']).reshape(-1, 1)
model_a, model_p = LinearRegression().fit(x,y_a), LinearRegression().fit(x,y_p)
df['a_trend'] = model_a.intercept_[0] + model_a.coef_[0][0]*df['RelDate']
df['p_trend'] = model_p.intercept_[0] + model_p.coef_[0][0]*df['RelDate']
axs[0].plot_date(df['Date'].dt.date, df['a_trend'], '--', color = 'darkgreen', alpha = 0.5)
axs[1].plot_date(df['Date'].dt.date, df['p_trend'], '--', color = 'darkgreen', alpha = 0.5)
axs[0].text(-0.08, -0.0, 'a', transform=axs[0].transAxes, size=12, weight='bold')
axs[1].text(-0.08, -0.0, 'b', transform=axs[1].transAxes, size=12, weight='bold')

print('Acropora slope: {:.0f} liters/year'.format(model_a.coef_[0][0]*365.25))
print('Pocillopora slope: {:.0f} liters/year'.format(model_p.coef_[0][0]*365.25))

# Methods - Residuals

In [ ]:
from scipy.stats import norm

df = pd.read_csv('SizeCheck.csv')
df['Len_res'] = df['mL'] - 0.3296 * df['cL']
df['Vol_res'] = df['mV'] - 0.1568*df['cV']

In [ ]:
fig, axs = pyplot.subplots(1,2, figsize = [12, 5])

df.hist('Len_res', ax = axs[0], density = True, bins = 50)
x0 = np.arange(-3, 3, 0.001)
y0 = norm.pdf(x0,df['Len_res'].mean(),df['Len_res'].std())
axs[0].set(xlabel='Length residuals (centimeters)')
axs[0].set(ylabel='Frequency')
axs[0].set(title=None)
axs[0].plot(x0, y0)
axs[0].text(-0.05, -0.11, 'a', transform=axs[0].transAxes, size=12, weight='bold')

df.hist('Vol_res', ax = axs[1], density = True, bins = 50)
x1 = np.arange(-600, 600, 0.1)
y1 = norm.pdf(x1,df['Vol_res'].mean(),df['Vol_res'].std())
axs[1].set(xlabel='Volume residuals (cube centimeters)')
axs[1].set(ylabel='Frequency')
axs[1].set(title=None)
axs[1].plot(x1, y1)
axs[1].text(-0.05, -0.11, 'b', transform=axs[1].transAxes, size=12, weight='bold')